In [2]:
import requests
import pandas as pd
import numpy as np
import json
from sklearn import preprocessing

In [14]:
logo_urls = pd.read_excel('logo_urls.xlsx'); logo_urls

,brand,url
0,8seconds,https://s3.namuwikiusercontent.com/s/a5305313f...
1,adidas,http://i0.wp.com/s3.store.hypebeast.com/media/...
2,adidasoriginal,http://i0.wp.com/s3.store.hypebeast.com/media/...
3,asics,http://i0.wp.com/s3.store.hypebeast.com/media/...
4,balenciaga,http://i0.wp.com/s3.store.hypebeast.com/media/...
5,bally,https://upload.wikimedia.org/wikipedia/en/thum...
6,beanpole,https://s3.namuwikiusercontent.com/s/08d356c19...
7,benetton,https://upload.wikimedia.org/wikipedia/commons...
8,blackyak,https://scontent-icn1-1.xx.fbcdn.net/v/t1.0-9/...
9,buckaroo,https://scontent-icn1-1.xx.fbcdn.net/v/t1.0-9/...


In [17]:
for row in logo_urls.itertuples():
    if row.brand not in ['beanpole','chanel','diesel','jansport','lafuma']:
        continue
        
    img = requests.get(row.url).content
    with open('app/static/logos/' + row.brand + '.jpg', 'wb') as f:
        f.write(img)

In [12]:
url = 'https://t1tallinn.com/wp-content/uploads/2018/10/MandarinaDuck_T1.jpg'
img = requests.get(url).content
with open('test.jpg', 'wb') as f:
    f.write(img)

In [29]:
df = pd.read_pickle('id_dict.pkl')
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0.1, 1))
X_train_minmax = min_max_scaler.fit_transform(df)
df[:] = X_train_minmax

def get_identity(bname):
    try:
        _df = df[[bname]].reset_index().rename(columns={'index':'key', bname:'value'})
        _df.value = (_df.value*100).astype(int)
        return _df.to_dict('record')
    except:
        return None

In [31]:
db_init = []
for i, brand in enumerate(pd.read_excel('keywords.xlsx', header=1).item[3:]):
    db_init.append({
        'model': 'app.brand', 
        'pk': i, 
        'fields': {
            'name': brand, 
            'logo': brand + '.jpg', 
            'identity': json.dumps(get_identity(brand))
        }
    })

In [33]:
with open('db_init.json', 'w') as f:
    json.dump(db_init, f)